In [16]:
import os
import json
import requests
from IPython.display import update_display,Markdown,display
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from openai import OpenAI
import anthropic
import google.generativeai
import gradio as gr

In [17]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCp


In [11]:
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

In [13]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [20]:
def chat(message,history):
    messages= [{'role':'system','content':system_message}]
    for human, assistant, *rest in history:
        messages.append({'role':'user','content':human})
        messages.append({'role':'assistant','content':assistant})
    messages.append({'role':'user','content':message})

    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages
    )

    return response.choices[0].message.content

In [21]:
gr.ChatInterface(fn = chat).launch(share=True)

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://1b6565484d6ca7c202.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Define Function 

In [52]:
ticket_prices = {
    "london": "$799", 
    "paris": "$899", 
    "tokyo": "$1400", 
    "berlin": "$499"
}

In [56]:
def get_ticket_price(destination_city):
    print(f"Tool to get_ticket_price for {destination_city} is")
    city = destination_city.lower()
    return ticket_prices.get(city,"Unknown")

In [58]:
price_function = {
    'name':'get_ticket_price',
    'description':"Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    'parameters': {
        'type':'object',
        'properties': {
            'destination_city': {
                'type':'string',
                'description':"The city that the customer wants to travel to"
            }
        },
        'required' : ['destination_city'],
        'additionalProperties' : False
    }
}

In [60]:
tools = [{'type':'function','function':price_function}]

In [81]:
def chat(message,history):
    messages = [{'role':'system','content':system_message}]
    for human, assistant , *rest in history:
        messages.append({'role':'user','content':human})
        messages.append({'role':'assistant','content':assistant})
    messages.append({'role':'user','content':message})

    response = openai.chat.completions.create(
        messages = messages,
        model = 'gpt-4o-mini',
        tools = tools
    )

    if response.choices[0].finish_reason == 'tool_calls':
       message = response.choices[0].message
       response , city = handle_tool_call(message)
       messages.append(message)
       messages.append(response)
       response = openai.chat.completions.create(
           model = 'gpt-4o-mini',
           messages = messages
       )

    return response.choices[0].message.content

In [82]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        'role':'tool',
        'content':json.dumps({'destination_city':city,'price':price}),
        'tool_call_id' : message.tool_calls[0].id
    }
    return response, city

In [85]:
gr.ChatInterface(fn = chat,type = 'messages').launch()

* Running on local URL:  http://127.0.0.1:7872
* To create a public link, set `share=True` in `launch()`.
